## Vector store-backed memory
docs: https://python.langchain.com/docs/modules/memory/how_to/vectorstore_retriever_memory

In [2]:
from datetime import datetime
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

import faiss
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS, Pinecone

# Initialize VectorStore
embeddings_size = 1536  # OpenAIEmbeddings 的维度
index = faiss.IndexFlatL2(embeddings_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectore_store = FAISS(embedding_fn, index, InMemoryDocstore({}), {})


In [5]:
# create VectorStoreRetrieverMemory
retriever = vectore_store.as_retriever(search_kwargs=dict(k=1)) # 实际应用中k要是设大点
memory = VectorStoreRetrieverMemory(retriever=retriever)

# 当添加到代理时，内存对象可以保存来自对话或使用的工具的相关信息
memory.save_context({"input": "My favorite food is pizza"}, {"output": "thats good to know"})
memory.save_context({"input": "My favorite sport is soccer"}, {"output": "..."})
memory.save_context({"input": "I don't the Celtics"}, {"output": "ok"})

In [8]:
rs = memory.load_memory_variables({"prompt": "what sport should i watch?"})
print(rs)
print(rs['history'])


{'history': 'input: My favorite sport is soccer\noutput: ...'}
input: My favorite sport is soccer
output: ...


In [9]:
# use in a chain
llm = OpenAI(temperature=0)

_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""

prompt = PromptTemplate(
    input_variables=["input", "history"],
    template=_DEFAULT_TEMPLATE,
)

conversation_with_summary = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True
)

conversation_with_summary.predict(input='Hi, my name is Perry')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite food is pizza
output: thats good to know

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi, my name is Perry
AI:

> Finished chain.


" Hi Perry, my name is AI. It's nice to meet you. What do you like to do in your free time?"

In [10]:
conversation_with_summary.predict(input="what's my favorite sport?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite sport is soccer
output: ...

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: what's my favorite sport?
AI:

> Finished chain.


' You told me earlier that your favorite sport is soccer.'

In [11]:
conversation_with_summary.predict(input="Whats my favorite food")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite food is pizza
output: thats good to know

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Whats my favorite food
AI:

> Finished chain.


' You said your favorite food is pizza.'

In [12]:
conversation_with_summary.predict(input="What's my name?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi, my name is Perry
response:  Hi Perry, my name is AI. It's nice to meet you. What do you like to do in your free time?

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: What's my name?
AI:

> Finished chain.


' Your name is Perry.'

In [13]:
conversation_with_summary.predict(input="What did we talk about?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: what's my favorite sport?
response:  You told me earlier that your favorite sport is soccer.

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: What did we talk about?
AI:

> Finished chain.


" We talked about your favorite sport, which you said is soccer. We also discussed the weather and how it's been unseasonably warm lately."